In [ ]:
!wget http://www.cs.cmu.edu/~jiweil/review.txt.zip

--2024-04-17 22:12:55--  http://www.cs.cmu.edu/~jiweil/review.txt.zip
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373708772 (356M) [application/zip]
Saving to: ‘review.txt.zip’

review.txt.zip      100%[===================>] 356.40M  4.80MB/s    in 75s     

2024-04-17 22:14:11 (4.75 MB/s) - ‘review.txt.zip’ saved [373708772/373708772]



In [ ]:
!unzip review.txt.zip
!mkdir /content/data/tripadvisor
!mv review.txt /content/data/tripadvisor/review.txt

Archive:  review.txt.zip
  inflating: review.txt              


In [ ]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()

In [ ]:
path = '/content/data/tripadvisor/review.txt'
with open(path, 'r') as f:
    texts = f.readlines()

In [ ]:
dat = []
for i, t in enumerate(tqdm(texts)):
    t = json.loads(t)
    d = {
        'index': i,
        'hotel_id': t['id'],
        'user_id' : t['author']['id']
    }
    for key in ['title', 'text','date_stayed', 'date']:
        d[key] = t.get(key, np.nan)
    for key in ['overall', 'cleanliness', 'value', 'location', 'rooms', 'sleep_quality']:
        d[key] = t['ratings'].get(key, np.nan)
    dat.append(d)
dat = pd.DataFrame(dat)

for v in ['hotel_id', 'user_id', 'title', 'text', 'date_stayed', 'date']:
    dat = dat[dat[v]!='']
d = dat.copy()
d.sample(2)

100%|██████████| 878561/878561 [00:18<00:00, 48703.18it/s]


,index,hotel_id,user_id,title,text,date_stayed,date,overall,cleanliness,value,location,rooms,sleep_quality
236838,236838,11451393,8948B9B9891C96D4257245916107C739,"“Great Location, Good Hotel, Nice touches”",Stayed here over New Year 2006 / 2007. Excelle...,December 2006,"December 10, 2007",4.0,4.0,4.0,5.0,4.0,NaN
666266,666266,41717472,394511B559B016D4E9578D067E675984,“Loved it - and such a deal!”,We stayed at the Omni for Labor Day weekend an...,September 2009,"September 14, 2009",5.0,5.0,5.0,5.0,4.0,NaN


In [ ]:
!pip install -q fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
import fasttext
model = fasttext.load_model('lid.176.bin')

In [ ]:
d = dat.copy().set_index('index')
d.loc[d['date_stayed']=='','date_stayed'] = np.nan
d.loc[d['date_stayed'].isna(),'date_stayed'] = np.nan

d.loc[~d['date_stayed'].astype(str).str.contains(' '), 'date_stayed'] = np.nan
d = d.dropna(subset=['date_stayed'])

d['stay_year'] = d['date_stayed'].astype(str).apply(lambda x: x.split(' ')[1]).astype(int)
d['post_date'] = pd.to_datetime(d['date'], format='%B %d, %Y')
d['freq'] = 1
d['title'] = d['title'].str.replace("“",'')
d['title'] = d['title'].str.replace("”",'')
d['review'] = d['title'] + '\n' + d['text']
d['char'] = d['review'].str.len()

d = d.drop(columns=['date_stayed', 'date'])
d = d.dropna(subset=['overall','cleanliness','value','location','rooms','sleep_quality'])
d = d[d['stay_year']==2012].sample(frac=1).reset_index(drop=True)

# 英文のみ抽出
d['lang'] = d['text'].str.replace('\n','').progress_apply(lambda x: model.predict(x))
d['lang'] = d['lang'].apply(lambda x: x[0][0])
d = d[d['lang'].str.contains('__label__en')]

f = '/content/data/tripadvisor/data.pkl'
d.to_pickle(f)

100%|██████████| 218077/218077 [00:56<00:00, 3887.60it/s]


In [ ]:
display(d.sample(3))

,hotel_id,user_id,title,text,overall,cleanliness,value,location,rooms,sleep_quality,stay_year,post_date,freq,review,char,lang
0,127781101,2262DCBFC351F42A9DD30AC8BAD24686,Really excellent Hilton,Stayed here on business trips and the hotel is...,5.0,4.0,5.0,4.0,5.0,4.0,2012,2012-04-13,1,Really excellent Hilton\nStayed here on busine...,204,__label__en
1,137380592,8477E11DABF4D6743885E401BB4C8CCF,Exceptional service and comfort,Spent two nights here for a wedding in Brookly...,5.0,5.0,4.0,5.0,4.0,5.0,2012,2012-08-16,1,Exceptional service and comfort\nSpent two nig...,621,__label__en
2,129673371,483A193B7113ADFFD5CE30849564F69C,Nice room and five star service,Great place for a 3-night stay. Our king room ...,5.0,5.0,5.0,3.0,5.0,4.0,2012,2012-05-09,1,Nice room and five star service\nGreat place f...,1259,__label__en
3,129006626,E5A63DD7239A7057746D4644A5C986EB,"BRILLIANT hotel, my #1 Chicago pick for busine...","This is my favorite hotel in Chicago, and I've...",5.0,5.0,5.0,5.0,5.0,5.0,2012,2012-04-28,1,"BRILLIANT hotel, my #1 Chicago pick for busine...",2242,__label__en
4,139168159,CBFE281C9386225267BC52518836A6C2,Convenient and comfortable,BEST. BREAKFAST. EVER. Couldn't have been happ...,5.0,5.0,4.0,5.0,4.0,5.0,2012,2012-09-02,1,Convenient and comfortable\nBEST. BREAKFAST. E...,511,__label__en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218072,135811676,F343B7C4978A928494222A63EA33259B,Great find in cool neighborhood,"Although a native NY'er, I haven't spent any t...",4.0,4.0,4.0,5.0,4.0,4.0,2012,2012-07-30,1,Great find in cool neighborhood\nAlthough a na...,1545,__label__en
218073,129553333,AF4C4D7F38A7CB053D226AE7954294B7,Exceptional Service and great room,I have stayed here a couple of times and have ...,5.0,5.0,5.0,5.0,5.0,5.0,2012,2012-05-07,1,Exceptional Service and great room\nI have sta...,230,__label__en
218074,143681514,C88ACB857117608D118C91D56F79531A,Beware of the Rip Off!,I received a call for a two day stay at the Hi...,2.0,2.0,1.0,4.0,2.0,1.0,2012,2012-10-25,1,Beware of the Rip Off!\nI received a call for ...,572,__label__en
218075,139622044,39EE66822F41168E3867C9DBED81081A,It Deserves the Rating,"I really like Kimptons. Every so often, you'll...",5.0,5.0,4.0,5.0,5.0,5.0,2012,2012-09-07,1,It Deserves the Rating\nI really like Kimptons...,746,__label__en
